In [3]:
from segmentation import segment, init_segment_model
from augmentation import augment, init_augment_model
import SimpleITK as sitk
import os
import csv
import numpy as np
from util import XyzTuple, xyz2irc, logging, getCache
import collections
from collections import namedtuple  
import glob
from pylidc_func import masks_build
from make_mask import aug_mask
import torch
from skimage import measure, filters
import scipy.ndimage.morphology

In [4]:
def threshold_mask(raw_output, threshold):
    if threshold == 0:
        try:
            threshold = filters.threshold_otsu(raw_output)
        except:
            threshold = 0.5

    # print('\tThreshold: {}'.format(threshold))

    raw_output[raw_output > threshold] = 1
    raw_output[raw_output < 1] = 0

    all_labels = measure.label(raw_output)
    props = measure.regionprops(all_labels)
    props.sort(key=lambda x: x.area, reverse=True)
    thresholded_mask = np.zeros(raw_output.shape)

    if len(props) >= 2:
        if props[0].area / props[1].area > 5:  # if the largest is way larger than the second largest
            thresholded_mask[all_labels == props[0].label] = 1  # only turn on the largest component
        else:
            thresholded_mask[all_labels == props[0].label] = 1  # turn on two largest components
            thresholded_mask[all_labels == props[1].label] = 1
    elif len(props):
        thresholded_mask[all_labels == props[0].label] = 1

    thresholded_mask = scipy.ndimage.morphology.binary_fill_holes(thresholded_mask).astype(np.uint8)

    return thresholded_mask

In [11]:
uid = "1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260"
# 1.3.6.1.4.1.14519.5.2.1.6279.6001.267957701183569638795986183786_179_qual_fig.png
imagePath = "D://LUNA//Luna16_AugSegData//subset0//1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260.mhd"
# mask_path = "C://LUNA//udet//result//output//1.3.6.1.4.1.14519.5.2.1.6279.6001.675543413149938600000570588203_75.pth"
ct_mhd = sitk.ReadImage(imagePath)
if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
    ct_mhd = ct_mhd[...,0]
origin = sitk.GetArrayFromImage(ct_mhd)
# masks = masks_build(uid, origin)
# loaded_data = torch.load(mask_path)
# prediction_g = loaded_data['prediction_g'].squeeze(1)
# mask = (torch.from_numpy(threshold_mask(prediction_g.cpu(), 0.5)).to("cuda"))
# augmented_mask = aug_mask(masks)

In [12]:
for i, image in enumerate(origin):
    np2Png(image.astype(float), "./lung/{}_{}.png".format(i, uid))

In [6]:
uid = "1.3.6.1.4.1.14519.5.2.1.6279.6001.675543413149938600000570588203"
# 1.3.6.1.4.1.14519.5.2.1.6279.6001.267957701183569638795986183786_179_qual_fig.png
imagePath = "D://LUNA//Luna16_SegData//subset8//1.3.6.1.4.1.14519.5.2.1.6279.6001.675543413149938600000570588203.mhd"
mask_path = "C://LUNA//udet//result//output//1.3.6.1.4.1.14519.5.2.1.6279.6001.675543413149938600000570588203_75.pth"
ct_mhd = sitk.ReadImage(imagePath)
if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
    ct_mhd = ct_mhd[...,0]
origin = sitk.GetArrayFromImage(ct_mhd)
# masks = masks_build(uid, origin)
loaded_data = torch.load(mask_path)
prediction_g = loaded_data['prediction_g'].squeeze(1)
mask = (torch.from_numpy(threshold_mask(prediction_g.cpu(), 0.5)).to("cuda"))
# augmented_mask = aug_mask(masks)

C:\Users\oplab\AppData\Local\Temp\ipykernel_18332\2846807596.py:27: DeprecationWarning: Please use `binary_fill_holes` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  thresholded_mask = scipy.ndimage.morphology.binary_fill_holes(thresholded_mask).astype(np.uint8)


In [10]:
uid = "1.3.6.1.4.1.14519.5.2.1.6279.6001.267957701183569638795986183786"
# 1.3.6.1.4.1.14519.5.2.1.6279.6001.267957701183569638795986183786_179_qual_fig.png
imagePath = "D://LUNA//Luna16_SegData//subset2//1.3.6.1.4.1.14519.5.2.1.6279.6001.267957701183569638795986183786.mhd"
mask_path = "C://LUNA//udet//result//output//1.3.6.1.4.1.14519.5.2.1.6279.6001.267957701183569638795986183786_179.pth"
ct_mhd = sitk.ReadImage(imagePath)
if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
    ct_mhd = ct_mhd[...,0]
origin = sitk.GetArrayFromImage(ct_mhd)
# masks = masks_build(uid, origin)
loaded_data = torch.load(mask_path)
prediction_g = loaded_data['prediction_g'].squeeze(1)
mask = (torch.from_numpy(threshold_mask(prediction_g.cpu(), 0.5)).to("cuda"))
# augmented_mask = aug_mask(masks)

C:\Users\oplab\AppData\Local\Temp\ipykernel_18332\2846807596.py:27: DeprecationWarning: Please use `binary_fill_holes` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  thresholded_mask = scipy.ndimage.morphology.binary_fill_holes(thresholded_mask).astype(np.uint8)


In [7]:
from PIL import Image, ImageDraw
def np2Png(np_arr, target_path):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr)
    # d = 4000 if max_value != 1 else 1
    d = max_value - min_value
    scaled_np_arr = (np_arr - min_value) / (d) * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slices = Image.fromarray(scaled_np_arr, mode='L')
    slices.save(target_path)

In [16]:
mask.size()

torch.Size([1, 512, 512])

In [9]:
np2Png(mask[0].cpu().numpy().astype(float), "./mask/demo/mask1.png")
np2Png(origin[75], "./mask/demo/origin1.png")
ori_mask = np.where(mask[0].cpu().numpy(), origin[75], 1000)
np2Png(ori_mask, "./mask/demo/ori_mask1.png")

In [11]:
np2Png(mask[0].cpu().numpy().astype(float), "./mask/demo/mask2.png")
np2Png(origin[179], "./mask/demo/origin2.png")
ori_mask = np.where(mask[0].cpu().numpy(), origin[179], 1000)
np2Png(ori_mask, "./mask/demo/ori_mask2.png")

In [25]:
mask[0].cpu().numpy()

array([[2.7818473e-07, 5.2934048e-08, 3.6602476e-08, ..., 4.6312248e-08,
        6.0776529e-08, 7.1300400e-07],
       [3.1144033e-08, 2.8228462e-09, 1.4234803e-09, ..., 2.0339335e-09,
        4.2193427e-09, 1.2466751e-07],
       [1.6372388e-08, 1.3398861e-09, 5.4652544e-10, ..., 4.6388760e-10,
        1.9923467e-09, 1.0676341e-07],
       ...,
       [4.4270166e-08, 4.5827093e-09, 2.1230437e-09, ..., 1.2686754e-09,
        4.6175774e-09, 1.2856430e-07],
       [6.6431156e-08, 7.0193629e-09, 4.5756963e-09, ..., 4.2060204e-09,
        5.5903180e-09, 5.6670697e-08],
       [5.0730830e-07, 9.9064856e-08, 6.2594765e-08, ..., 1.0992514e-07,
        4.6424041e-08, 3.9948949e-07]], dtype=float32)

In [2]:
CandidateInfoTuple = namedtuple('CandidateInfoTuple', 'isNodule_bool, hasAnnotation_bool, isMal_bool, diameter_mm, series_uid, center_xyz')
def getCandidateInfoList(requireOnDisk_bool=True):
    # We construct a set with all series_uids that are present on disk.
    # This will let us use the data, even if we haven't downloaded all of
    # the subsets yet.
    mhd_list = glob.glob('C:/Users/oplab/Desktop/Luna16_data/Luna16_img/subset*/*.mhd')
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

    candidateInfo_list = []
    with open('C:/Users/oplab/Desktop/Luna16_data/annotations_with_malignancy.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm = float(row[4])
            isMal_bool = {'False': False, 'True': True}[row[5]] #it record the malignancy or not

            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            candidateInfo_list.append(
                CandidateInfoTuple(
                    True,
                    True,
                    isMal_bool,
                    annotationDiameter_mm,
                    series_uid,
                    annotationCenter_xyz,
                )
            )

    with open('C:/Users/oplab/Desktop/Luna16_data/candidates_V2.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]

            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            isNodule_bool = bool(int(row[4]))
            candidateCenter_xyz = tuple([float(x) for x in row[1:4]])

            if not isNodule_bool:
                candidateInfo_list.append(
                    CandidateInfoTuple(
                        False,
                        False,
                        False,
                        0.0,
                        series_uid,
                        candidateCenter_xyz,
                    )
                )

    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list

In [3]:
def getCandidateInfoDict(requireOnDisk_bool=True):  #把candidateInfoList包成Dict
    candidateInfo_list = getCandidateInfoList(requireOnDisk_bool)
    candidateInfo_dict = {}

    for candidateInfo_tup in candidateInfo_list:
        candidateInfo_dict.setdefault(candidateInfo_tup.series_uid,
                                      []).append(candidateInfo_tup)

    return candidateInfo_dict

In [4]:
# candidate_dict =  getCandidateInfoDict()

In [5]:
# write_root = "D://LUNA//Luna16_AugData"
# if not os.path.exists(write_root):
#     os.mkdir("D://LUNA//Luna16_AugData")
    
# append = True
# # split to train test
# for sub_root in os.listdir("C://Users//oplab//Desktop//Luna16_data//Luna16_img//"):
#     root = os.path.join("E:\LUNA\Luna_Data", sub_root)
#     print(root)
#     for path in os.listdir(root):
#         break
#         # npy_path = glob.glob('./augmented_segmented/{}.npy'.format(series_uid))
#         npy_path = glob.glob(os.path.join(write_root, sub_root, path) + ''.format(series_uid))
#         if (npy_path != [] and append):
#             print("pass")
#             continue
#         mhd_path = glob.glob('C:/Users/oplab/Desktop/Luna16_data/Luna16_img/subset*/{}.mhd'.format(series_uid))
#         ct_mhd = sitk.ReadImage(mhd_path)
#         if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
#             ct_mhd = ct_mhd[...,0]
#         origin = sitk.GetArrayFromImage(ct_mhd)
#         print(mhd_path)
#         augmented = augment(origin)
#         # segmented = segment(augmented)
#         # hu_a = np.array(segmented, dtype=np.float32)
#         hu_a = np.array(augmented, dtype=np.float32)
#         # np.save("./augmented/{}.npy".format(series_uid), hu_a)
#         # np.save("./augmented_segmented/{}.npy".format(series_uid), hu_a)
#         # Load or create a SimpleITK image
#         current_origin = ct_mhd.GetOrigin()
#         current_spacing = ct_mhd.GetSpacing()

#         sitk_image = sitk.GetImageFromArray(augmented)
#         # image_short = sitk.Cast(image, sitk.sitkInt16)

#         # Set the image origin, spacing, and direction (modify as needed)
#         sitk_image.SetOrigin((current_origin[0], current_origin[1], current_origin[2]))
#         sitk_image.SetSpacing((current_spacing[0], current_spacing[1], current_spacing[2] / 2))

#         # Save the image as a MetaImage file
#         if not os.path.exists(os.path.join(write_root, sub_root)):
#             os.mkdir(os.path.join(write_root, sub_root))
#         sitk.WriteImage(sitk_image, os.path.join(write_root, sub_root, path)) 

In [6]:
# npy_path = glob.glob(os.path.join(write_root, sub_root, path) + ''.format(series_uid))

In [7]:
model_best, device = init_augment_model()
model, device = init_segment_model()

In [ ]:
write_root_1 = "D:\\LUNA\\Luna16_AugData"
if not os.path.exists(write_root_1):
    os.mkdir("D:\\LUNA\\Luna16_AugData")
write_root_2 = "D:\\LUNA\\Luna16_SegData"
if not os.path.exists(write_root_2):
    os.mkdir("D:\\LUNA\\Luna16_SegData")
    
# read_root = "C:\\Users\\oplab\\Desktop\\Luna16_data\\Luna16_img"
read_root = "D:\\LUNA\\Luna16_AugData"
    
append = True

# split to train test
for sub_root in os.listdir(read_root):
    root = os.path.join(read_root, sub_root)
    print(root)
    for path in os.listdir(root):
        if path.find("mhd") >= 0:
            raw_path_1 = os.path.join(write_root_1, sub_root, path)
            exist_path_1 =  glob.glob(os.path.join(write_root_1, sub_root, path))
            raw_path_2 = os.path.join(write_root_2, sub_root, path)
            exist_path_2 =  glob.glob(os.path.join(write_root_2, sub_root, path))
            if (exist_path_1 != [] and exist_path_2 != [] and append):
                print("pass")
                continue
            print(path)
            imagePath = os.path.join(root, path)
            ct_mhd = sitk.ReadImage(imagePath)
            if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
                ct_mhd = ct_mhd[...,0]
            origin = sitk.GetArrayFromImage(ct_mhd)
#             augmented = augment(origin, model_best, device)
#             hu_a = np.array(augmented, dtype=np.float32)
#             # Load or create a SimpleITK image
#             current_origin = ct_mhd.GetOrigin()
#             current_spacing = ct_mhd.GetSpacing()

#             sitk_image = sitk.GetImageFromArray(augmented)
#             # image_short = sitk.Cast(image, sitk.sitkInt16)

#             # Set the image origin, spacing, and direction (modify as needed)
#             sitk_image.SetOrigin((current_origin[0], current_origin[1], current_origin[2]))
#             sitk_image.SetSpacing((current_spacing[0], current_spacing[1], current_spacing[2] / 2))

#             # Save the image as a MetaImage file
#             if not os.path.exists(os.path.join(write_root_1, sub_root)):
#                 os.mkdir(os.path.join(write_root_1, sub_root))
#             print(os.path.join(write_root_1, sub_root, path))
#             sitk.WriteImage(sitk_image, raw_path_1) 
            
           # segment part
            segmented = segment(origin, model, device)
            hu_a = np.array(segmented, dtype=np.float32)
            # Load or create a SimpleITK image
            current_origin = ct_mhd.GetOrigin()
            current_spacing = ct_mhd.GetSpacing()

            sitk_image = sitk.GetImageFromArray(segmented)
            # image_short = sitk.Cast(image, sitk.sitkInt16)

            # Set the image origin, spacing, and direction (modify as needed)
            sitk_image.SetOrigin((current_origin[0], current_origin[1], current_origin[2]))
            sitk_image.SetSpacing((current_spacing[0], current_spacing[1], current_spacing[2]))

            # Save the image as a MetaImage file
            if not os.path.exists(os.path.join(write_root_2, sub_root)):
                os.mkdir(os.path.join(write_root_2, sub_root))
            print(os.path.join(write_root_2, sub_root, path))
            sitk.WriteImage(sitk_image, raw_path_2) 

D:\LUNA\Luna16_AugData\subset0
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
1.3.6.1.4.1.14519.5.2.1.6279.6001.640729228179368154416184318668.mhd
D:\LUNA\Luna16_SegData\subset0\1.3.6.1.4.1.14519.5.2.1.6279.6001.640729228179368154416184318668.mhd
1.3.6.1.4.1.14519.5.2.1.6279.6001.657775098760536289051744981056.mhd
D:\LUNA\Luna16_SegData\subset0\1.3.6.1.4.1.14519.5.2.1.6279.6001.657775098760536289051744981056.mhd
1.3.6.1.4.1.14519.5.2.1.6279.6001.716498695101447665580610403574.mhd
D:\LUNA\Luna16_SegData\subset0\1.3.6.1.4.1.14519.5.2.1.6279.6001.716498695101447665580610403574.mhd
1.3.6.1.4.1.14519.5.2.1.6279.6001.724251104254976962355686318345.mhd
D:\LUNA\Luna1

In [ ]:
print(raw_path)

In [ ]:
os.path.join(write_root, sub_root, path)

In [ ]:
glob.glob(os.path.join(write_root, sub_root, path))

In [ ]:
sitk.WriteImage(sitk_image, "D://udet//augmented//{}.raw".format(series_uid)) 

In [ ]:
npy_path = glob.glob('./augmented_segmented/{}.npy'.format("1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886"))

In [ ]:
npy_path

In [ ]:
if (npy_path != [] and append):
        print("pass")

In [ ]:
np.save("./augmented_segmented/{}.npy".format(series_uid), hu_a)